In [173]:
import random
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.utils import shuffle


In [174]:
def generate_weights(num_layers, input_size, num_nodes):
    weight_matrices = []

    # Creating the initial_layer
    initial_layer = [[0] * input_size for _ in range(num_nodes)]
    weight_matrices.append(initial_layer)

    # Creating the intermediate layers
    intermediate_layer = [[0] * num_nodes for _ in range(num_nodes)]
    for _ in range(num_layers - 2):
        weight_matrices.append(intermediate_layer)

    return weight_matrices


In [175]:
def generate_NN(num_neurons_hidden, num_neurons_output, num_input_samples):
    input_nodes = [[0] * 3 for _ in range(num_input_samples)]
    hidden_nodes = [[0] * 3 for _ in range(num_neurons_hidden)]
    output_nodes = [[0] * 3 for _ in range(num_neurons_output)]
    return [input_nodes, hidden_nodes, output_nodes]


In [176]:
def sigmoid_activation(x):
    return 1 / (1 + np.exp(-x))


In [177]:
def forward_propagation(weights, nodes):
    for current_layer_index in range(len(nodes) - 1):
        for current_node_index in range(len(nodes[current_layer_index + 1])):
            total_weighted_sum = 0
            for i in range(len(nodes[current_layer_index])):
                total_weighted_sum += nodes[current_layer_index][i][1] * weights[current_layer_index][current_node_index][i]
            nodes[current_layer_index + 1][current_node_index][0] = total_weighted_sum
            nodes[current_layer_index + 1][current_node_index][1] = sigmoid_activation(nodes[current_layer_index + 1][current_node_index][0])
    return nodes


In [178]:
def backpropagation(input_nodes, weights, target, learning_rate):
    for current_layer in range(len(input_nodes) - 1, -1, -1):
        for current_node in range(len(input_nodes[current_layer])):
            if current_layer == len(input_nodes) - 1:
                sigma = input_nodes[current_layer][current_node][1] * (1 - input_nodes[current_layer][current_node][1]) * (target[current_node] - input_nodes[current_layer][current_node][1])
                input_nodes[current_layer][current_node][2] = sigma
                if current_layer != 0:
                    for prev_node in range(len(input_nodes[current_layer - 1])):
                        delta_weight = learning_rate * input_nodes[current_layer - 1][prev_node][1] * sigma
                        weights[current_layer - 1][current_node][prev_node] += delta_weight
            else:
                weighted_sum = sum(input_nodes[current_layer + 1][k][2] * weights[current_layer][k][current_node] for k in range(len(input_nodes[current_layer + 1])))
                sigma = input_nodes[current_layer][current_node][1] * (1 - input_nodes[current_layer][current_node][1]) * weighted_sum
                input_nodes[current_layer][current_node][2] = sigma
                if current_layer != 0:
                    for prev_node in range(len(input_nodes[current_layer - 1])):
                        delta_weight = learning_rate * input_nodes[current_layer - 1][prev_node][1] * sigma
                        weights[current_layer - 1][current_node][prev_node] += delta_weight

    return weights


In [179]:
iris = datasets.load_iris()
iris_X = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_Y = pd.get_dummies(iris.target)
iris_X_list=iris_X.values.tolist()
iris_Y_list=iris_Y.values.tolist()


In [180]:
# mnist = fetch_openml('mnist_784')

# # Convert to DataFrame and normalize the pixel values
# mnist_X = pd.DataFrame(mnist.data / 255.0)
# mnist_Y = pd.get_dummies(mnist.target)
# mnist_data = pd.concat([mnist_X, mnist_Y], axis=1)

# # Shuffle the concatenated data
# mnist_data_shuffled = shuffle(mnist_data)

# # Take the first 1000 samples after shuffling
# mnist_data_subset = mnist_data_shuffled.head(1000)

# # Separate features and target variable after shuffling
# mnist_X_subset = mnist_data_subset.iloc[:, :784]  # Assuming 784 is the number of features
# mnist_Y_subset = mnist_data_subset.iloc[:, 784:]
# mnist_X_list = mnist_X_subset.values.tolist()
# mnist_Y_list = mnist_Y_subset.values.tolist()
#len(mnist_X_list)

In [181]:
input_data = iris_X_list
target_data = iris_Y_list

learning_rate = 3
num_layers = 3
y_prid_list = []
weights = generate_weights(num_layers, len(input_data[0]), len(target_data[0]))
#print(weights)
for input_sample, target in zip(input_data, target_data):

    nodes = generate_NN(len(target), len(target), len(input_sample))

    for k,inp in zip( nodes[0],input_sample):
        k[1] = inp

    for i in range(20):
        nodes = forward_propagation(weights, nodes)
        weights = backpropagation(nodes, weights, target, learning_rate)

    y_list = [nodes[2][node][1] for node in range(len(nodes[2]))]
    y_prid_list.append(y_list)

columns = [f"col{i}" for i in range(1, len(y_prid_list[0]) + 1)]
rounded_pred_list = [[int(round(value)) for value in row] for row in y_prid_list]
predicted_output = pd.DataFrame(rounded_pred_list, columns=columns)

#accuracy = accuracy_score(mnist_Y_subset[0:1000], predicted_output)
accuracy = accuracy_score(iris_Y, predicted_output)
print("Accuracy Score:", accuracy * 100, "%")


Accuracy Score: 99.33333333333333 %
